<a href="https://colab.research.google.com/github/psandoval123/MetNum2024I/blob/main/Lab7/psandoval_Lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [ ]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
# ... ENTER YOUR CODE HERE ...
def jacobi(A, b, num_it=1000):
    n = A.shape[0]
    diag_A = np.diag(A)
    B = np.zeros_like(A)

    for i in range(n):
        for j in range(n):
            if i != j:
                B[i, j] = -A[i, j] / diag_A[i]

    c = b / diag_A
    norm = np.linalg.norm(B)

    x0 = np.ones(n)
    x = x0
    for _ in range(num_it):
        x = B @ x + c

    return x, norm


jacobi(A, b)


(array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
        -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648]),
 0.36436161983015336)

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
import matplotlib.pyplot as plt

def gauss_seidel(A, b, x_0, tol, max_iter):

  x = x_0.copy()

  for i in range(max_iter):

    for j in range(A.shape[0]):
      x[j] = (b[j] - np.dot(A[j, :j], x[:j])) / A[j, j]

    norm_x = np.linalg.norm(x - x_0)
    if norm_x < tol:
      break

  return x

A = np.random.rand(10, 10)
b = np.random.rand(10)
print(A)
x = gauss_seidel(A, b, np.zeros(10), 1e-6, 100)

true_x = np.linalg.solve(A, b)

error = np.linalg.norm(x - true_x)
print(error)

error_list = []
for i in range(1, 100):
  x = gauss_seidel(A, b, np.zeros(10), 1e-6, i)
  error_list.append(np.linalg.norm(x - true_x))


[[0.34187116 0.45376824 0.31791091 0.72503365 0.45974978 0.91129982
  0.36122294 0.1005722  0.78755274 0.59222187]
 [0.72408235 0.19423426 0.92865247 0.8061134  0.82761295 0.59195925
  0.71845798 0.86061365 0.91459643 0.32205375]
 [0.82992399 0.84372723 0.6708093  0.60304153 0.14316406 0.1873763
  0.7805792  0.25007811 0.87005774 0.63280404]
 [0.95352256 0.97765335 0.86715816 0.58569286 0.50520347 0.39647049
  0.36690623 0.43794065 0.63916442 0.64616292]
 [0.8575902  0.15106798 0.02350704 0.45931499 0.19834969 0.20470058
  0.47393652 0.46787272 0.40131357 0.75368993]
 [0.40815513 0.9357577  0.16440515 0.21487474 0.83117971 0.30568327
  0.70673948 0.5157713  0.4510183  0.43041778]
 [0.0621954  0.05997763 0.5558661  0.71655587 0.13183341 0.13372031
  0.99142568 0.95311723 0.38354378 0.71540915]
 [0.40845653 0.65240041 0.0382623  0.6275028  0.4027578  0.76120779
  0.99597245 0.01922291 0.32192955 0.84181168]
 [0.55237096 0.91138021 0.33269067 0.06410911 0.38393073 0.57963755
  0.11205703 

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
import numpy as np

def minres(A, b, n_iter=500, tol=1e-07):
    x = np.zeros_like(b)
    r = b - A @ x
    p = r.copy()
    T = []

    for i in range(n_iter):
        Ap = A @ p
        alpha = np.dot(r, r) / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        beta = np.dot(r, r) / np.dot(p, Ap)
        p = r + beta * p
        T.append(alpha)

        if np.linalg.norm(r) < tol:
            break

    return x, T


x, T = minres(A, b)

print("Solución:")
print(x)
print("Paramentos de iteracion: ")
print(T)



#Pruebas para el vector solucion
rtol = 1e-5
atol = 1e-5


xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@x, b, rtol=rtol, atol=atol)
np.testing.assert_allclose(x, xx, rtol=rtol, atol=atol)

Solución:
[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
Paramentos de iteracion: 
[0.052753954820858144, 0.06218738529697278, 0.05417593975731786, 0.061506362422169426, 0.05530993839707102, 0.060437341848485995, 0.055466488812815795, 0.06048091963713317, 0.05531708911926293]
